[View in Colaboratory](https://colab.research.google.com/github/from81/omis120stockmarket/blob/master/blockchain.ipynb)

In [1]:
import hashlib as hasher
import datetime as date
import pandas as pd

First, we need to define what a "block" is.<br>Another words, we will define the data structure of a block, and create a class object Block. Let's list what we need to store in each block:

<ul>
<li>index</li>
<li>timestamp (of generation)</li>
<li>data (whatever we want)</li>
<li>self-identifying hash</li>
</ul>

Like Bitcoin, each block’s hash will be a cryptographic hash of the block’s index, timestamp, data, 
and the hash of the previous block’s hash.

In [0]:
class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.hash_block()

    def hash_block(self):
        hsh = hasher.sha256()
        hsh.update(
            (
            str(self.index) + 
            str(self.timestamp) + 
            str(self.data) + 
            str(self.previous_hash)
            ).encode())
        return hsh.hexdigest()

We defined what each block will look like, and each block requires a preceding block.<br>
So our next step is to generate the first block (genesis block) which will precede all blocks thereafter:
<ul>
<li>Since this is just a demo, we will manually create a function that will output the genesis block</li>
<li>It will have index 0, an arbitrary data value, and an arbitrary value in the “previous hash” parameter</li>
</ul>

In [0]:
def genesis_block():
    return Block(0, date.datetime.now(), "Some data", "0")

So far, we have defined the data structure of a block, and also created a function to create the genesis block.<br>
Next, we will need to create a function that will generate the next blocks.<br>
Since we are making a blockCHAIN, this function will take the previous block as an input.<br>
Each block will not only record the hash of the previous block, but also incorporate that into the hash function<br>
By hashing recursively, the integrity and security of the blockchain increases as the length grows, which makes blockchain scalable.<br>

In [0]:
def next_block(last_block):
    this_index = last_block.index + 1
    this_timestamp = date.datetime.now()
    this_data = "Hey! I'm block " + str(this_index)
    this_hash = last_block.hash
    return Block(this_index, this_timestamp, this_data, this_hash)

Now, we can create our blockchain. We can begin this by calling the create_genesis_block() function that we defined earlier.

In [27]:
blockchain = [genesis_block()]
previous_block = blockchain[0]
print("Genesis block #{} created.".format(previous_block.index))
print("Hash: {}\n".format(previous_block.hash))

number_of_blocks = 20

for i in range(number_of_blocks):
    new_block = next_block(previous_block)
    blockchain.append(new_block)
    previous_block = new_block

    print("Block #{} has been added to the blockchain.".format(new_block.index))
    print("Hash: {}\n".format(new_block.hash))

Genesis block #0 created.
Hash: d99b1cac06ef7b5bc3f6a36d155beaacdff2c5c6da9efbaadc0b063a39e95870

Block #1 has been added to the blockchain.
Hash: 811d41cfc889f25416559bcb0974a8f1c013e50784f5d86e51fb6713140428ac

Block #2 has been added to the blockchain.
Hash: 421adb02286ee34f0a4f3962b1288d3e1e5940b447ec3c36fbd0ae3584534dcf

Block #3 has been added to the blockchain.
Hash: f494267774342a8b718067b9ad7eee5affcd5be18ba23ece44005da3fab549ed

Block #4 has been added to the blockchain.
Hash: e91fc03833be70f8072b5dfdd68c2ce13957c366b3b36391059745dfe67c43b7

Block #5 has been added to the blockchain.
Hash: a44be1caad774d3e23f8f19121dd664851eb280e38abb42e946c4d7a7cf3ed67

Block #6 has been added to the blockchain.
Hash: c4985cd4a8c33a6ed2a946c89900f4325f2b40e27c22bb900102ad61dffee0ff

Block #7 has been added to the blockchain.
Hash: 4e2463c6aab1d7ee24d20d8f8a636a80138413d5d78e6027c3e46b88f8bcaa1d

Block #8 has been added to the blockchain.
Hash: 7ed76fbed809a8fa314edf36a5a9b998e413780aabe6959f

In [0]:
chain = []
for block in blockchain:
    li = []
    li.append(block.index)
    li.append(block.timestamp)
    li.append(block.data)
    li.append(block.previous_hash)
    li.append(block.hash)
    chain.append(li)

In [0]:
blockchain = pd.DataFrame(chain, columns=['index', 'timestamp' ,'data' , 'previousHash', 'hash'])
blockchain = blockchain.set_index('index')

In [30]:
blockchain.head()

,timestamp,data,previousHash,hash
index,,,,
0,2018-06-28 10:52:14.525487,Some data,0,d99b1cac06ef7b5bc3f6a36d155beaacdff2c5c6da9efb...
1,2018-06-28 10:52:14.526690,Hey! I'm block 1,d99b1cac06ef7b5bc3f6a36d155beaacdff2c5c6da9efb...,811d41cfc889f25416559bcb0974a8f1c013e50784f5d8...
2,2018-06-28 10:52:14.527056,Hey! I'm block 2,811d41cfc889f25416559bcb0974a8f1c013e50784f5d8...,421adb02286ee34f0a4f3962b1288d3e1e5940b447ec3c...
3,2018-06-28 10:52:14.528375,Hey! I'm block 3,421adb02286ee34f0a4f3962b1288d3e1e5940b447ec3c...,f494267774342a8b718067b9ad7eee5affcd5be18ba23e...
4,2018-06-28 10:52:14.530023,Hey! I'm block 4,f494267774342a8b718067b9ad7eee5affcd5be18ba23e...,e91fc03833be70f8072b5dfdd68c2ce13957c366b3b363...


How can we know this is legit?<br>
Inside the Block class object, you will find the line `self.hash = self.hash_block()`<br>
This simply takes a string (in UTF-8 encoding) and uses that as a seed to generate a hash. Think of it as an encryption. That means, if you put together the index, timestamp, data, and previousHash of any given row above, you will be able to generate the same hash.<br>

Example:
At the time of testing, my first row is
*   index: 0
*   timestamp: 2018-06-28 10:52:14.525487
*   data: Some data
*   previousHash: 0
*   Hash: d99b1cac06ef7b5b



Therefore, if I go to https://passwordsgenerator.net/sha256-hash-generator/ and enter 
"02018-06-28 10:52:14.525487Some data0", I get the exact same hash.